In [4]:
!pip install ipython-sql

     |████████████████████████████████| 44 kB 4.6 MB/s eta 0:00:01
You should consider upgrading via the '/Users/dungnguyen/.pyenv/versions/3.10.2/bin/python3.10 -m pip install --upgrade pip' command.


In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import Base, Dataset, AssetClass, Region, User

# Create the database
engine = create_engine('sqlite:///test_database.db')
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Create sample data
commodities = AssetClass(name="Commodities")
equities = AssetClass(name="Equities")
real_estate = AssetClass(name="Real Estate")

north_america = Region(name="North America")
us = Region(name="US", parent=north_america)
canada = Region(name="Canada", parent=north_america)
europe = Region(name="Europe")

user1 = User(username="user1")
user2 = User(username="user2")

datasets = [
    Dataset(name="US Commodities", description="Commodity prices in the US", 
            asset_classes=[commodities], regions=[us], maintainers=[user1]),
    Dataset(name="Global Equities", description="World-wide stock prices", 
            asset_classes=[equities], regions=[north_america, europe], maintainers=[user2]),
    Dataset(name="NA Real Estate", description="Real estate data for North America", 
            asset_classes=[real_estate], regions=[us, canada], maintainers=[user1]),
    Dataset(name="Mixed Assets", description="Various asset types", 
            asset_classes=[commodities, equities], regions=[north_america], maintainers=[user1, user2])
]

# Add all objects to the session
session.add_all([commodities, equities, real_estate, north_america, us, canada, europe, user1, user2] + datasets)

# Commit the changes
session.commit()

print("Database created and populated successfully.")

# Close the session
session.close()

Database created and populated successfully.


In [8]:
%load_ext sql
%sql sqlite:///test_database.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [16]:
%%sql
EXPLAIN QUERY PLAN 
SELECT
    datasets.id AS datasets_id,
    datasets.name AS datasets_name,
    datasets.description AS datasets_description
FROM
    datasets
WHERE
    datasets.id IN (
        SELECT
            datasets.id
        FROM
            datasets
            JOIN dataset_region AS dataset_region_1 ON datasets.id = dataset_region_1.dataset_id
            JOIN regions ON regions.id = dataset_region_1.region_id
        WHERE
            regions.name = "North America"
    )
    AND (
        datasets.id NOT IN (
            SELECT
                datasets.id
            FROM
                datasets
                JOIN dataset_asset_class AS dataset_asset_class_1 ON datasets.id = dataset_asset_class_1.dataset_id
                JOIN asset_classes ON asset_classes.id = dataset_asset_class_1.asset_class_id
            WHERE
                asset_classes.name = "Real Estate"
        )
    )
    AND (
        datasets.id IN (
            SELECT
                datasets.id
            FROM
                datasets
                JOIN dataset_maintainer AS dataset_maintainer_1 ON datasets.id = dataset_maintainer_1.dataset_id
                JOIN users ON users.id = dataset_maintainer_1.user_id
            WHERE
                users.username = "user1"
        )
        OR datasets.id IN (
            SELECT
                datasets.id
            FROM
                datasets
                JOIN dataset_maintainer AS dataset_maintainer_2 ON datasets.id = dataset_maintainer_2.dataset_id
                JOIN users ON users.id = dataset_maintainer_2.user_id
            WHERE
                users.username = "user2"
        )
    );

   sqlite:///mytable.db
 * sqlite:///test_database.db
Done.


id,parent,notused,detail
2,0,0,SEARCH datasets USING INTEGER PRIMARY KEY (rowid=?)
6,0,0,LIST SUBQUERY 1
10,6,0,SCAN dataset_region_1
12,6,0,SEARCH datasets USING INTEGER PRIMARY KEY (rowid=?)
15,6,0,SEARCH regions USING INTEGER PRIMARY KEY (rowid=?)
33,0,0,LIST SUBQUERY 2
37,33,0,SCAN dataset_asset_class_1
39,33,0,SEARCH datasets USING INTEGER PRIMARY KEY (rowid=?)
42,33,0,SEARCH asset_classes USING INTEGER PRIMARY KEY (rowid=?)
64,0,0,LIST SUBQUERY 3


In [53]:
%%sql
EXPLAIN QUERY PLAN 
SELECT
    datasets.id AS datasets_id,
    datasets.name AS datasets_name,
    datasets.description AS datasets_description
FROM
    datasets
WHERE
    datasets.id IN (
        SELECT
            DISTINCT datasets.id
        FROM
            datasets
            JOIN dataset_region AS dataset_region_1 ON datasets.id = dataset_region_1.dataset_id
            JOIN regions ON regions.id = dataset_region_1.region_id
        WHERE
            regions.name = "North America"
    )
    AND (
        datasets.id NOT IN (
            SELECT
                DISTINCT datasets.id
            FROM
                datasets
                JOIN dataset_asset_class AS dataset_asset_class_1 ON datasets.id = dataset_asset_class_1.dataset_id
                JOIN asset_classes ON asset_classes.id = dataset_asset_class_1.asset_class_id
            WHERE
                asset_classes.name = "Real Estate"
        )
    )
    AND (
        datasets.id IN (
            SELECT
                DISTINCT datasets.id
            FROM
                datasets
                JOIN dataset_maintainer AS dataset_maintainer_1 ON datasets.id = dataset_maintainer_1.dataset_id
                JOIN users ON users.id = dataset_maintainer_1.user_id
            WHERE
                users.username = "user1"
        )
        OR datasets.id IN (
            SELECT
                DISTINCT datasets.id
            FROM
                datasets
                JOIN dataset_maintainer AS dataset_maintainer_2 ON datasets.id = dataset_maintainer_2.dataset_id
                JOIN users ON users.id = dataset_maintainer_2.user_id
            WHERE
                users.username = "user2"
        )
    );

   sqlite:///mytable.db
 * sqlite:///test_database.db
Done.


datasets_id,datasets_name,datasets_description
2,Global Equities,World-wide stock prices
4,Mixed Assets,Various asset types


In [26]:
%%sql
SELECT datasets.id AS datasets_id, datasets.name AS datasets_name, datasets.description AS datasets_description 
FROM datasets 
WHERE NOT EXISTS (SELECT 1 
FROM datasets d1 JOIN dataset_asset_class AS dataset_asset_class_1 ON datasets.id = dataset_asset_class_1.dataset_id JOIN asset_classes ON asset_classes.id = dataset_asset_class_1.asset_class_id 
WHERE asset_classes.name = "Commodities" AND datasets.id = d1.id)

   sqlite:///mytable.db
 * sqlite:///test_database.db
Done.


datasets_id,datasets_name,datasets_description
2,Global Equities,World-wide stock prices
3,NA Real Estate,Real estate data for North America


In [49]:
%%sql

SELECT datasets.id AS datasets_id, datasets.name AS datasets_name, datasets.description AS datasets_description 
FROM datasets
WHERE NOT (EXISTS (SELECT 1 
FROM dataset_asset_class AS dataset_asset_class_1 JOIN asset_classes ON asset_classes.id = dataset_asset_class_1.asset_class_id 
WHERE asset_classes.name = "Commodities" AND datasets.id = dataset_asset_class_1.dataset_id))


   sqlite:///mytable.db
 * sqlite:///test_database.db
Done.


datasets_id,datasets_name,datasets_description
2,Global Equities,World-wide stock prices
3,NA Real Estate,Real estate data for North America


In [51]:
%%sql
 SELECT datasets.id AS datasets_id, datasets.name AS datasets_name, datasets.description AS datasets_description 
FROM datasets 
WHERE NOT (EXISTS (SELECT 1 
FROM datasets JOIN dataset_asset_class AS dataset_asset_class_1 ON datasets.id = dataset_asset_class_1.dataset_id JOIN asset_classes ON asset_classes.id = dataset_asset_class_1.asset_class_id, dataset_asset_class 
WHERE asset_classes.name = "Commodities" AND datasets.id = dataset_asset_class.dataset_id))

   sqlite:///mytable.db
 * sqlite:///test_database.db
Done.


datasets_id,datasets_name,datasets_description


In [36]:
%%sql
SELECT EXISTS (SELECT 1 
FROM datasets d1 JOIN dataset_asset_class AS dataset_asset_class_1 ON datasets.id = dataset_asset_class_1.dataset_id JOIN asset_classes ON asset_classes.id = dataset_asset_class_1.asset_class_id 
WHERE asset_classes.name = "Commodities")

   sqlite:///mytable.db
 * sqlite:///test_database.db
(sqlite3.OperationalError) no such column: datasets.id
[SQL: SELECT EXISTS (SELECT 1 
FROM datasets d1 JOIN dataset_asset_class AS dataset_asset_class_1 ON datasets.id = dataset_asset_class_1.dataset_id JOIN asset_classes ON asset_classes.id = dataset_asset_class_1.asset_class_id 
WHERE asset_classes.name = "Commodities")]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [54]:
%%sql
EXPLAIN QUERY PLAN 
 SELECT datasets.id AS datasets_id, datasets.name AS datasets_name, datasets.description AS datasets_description 
FROM datasets 
WHERE (EXISTS (SELECT 1 
FROM datasets AS datasets_1 JOIN dataset_region AS dataset_region_1 ON datasets_1.id = dataset_region_1.dataset_id JOIN regions ON regions.id = dataset_region_1.region_id 
WHERE regions.name = "North America" AND datasets.id = datasets_1.id)) AND NOT (EXISTS (SELECT 1 
FROM datasets AS datasets_2 JOIN dataset_asset_class AS dataset_asset_class_1 ON datasets_2.id = dataset_asset_class_1.dataset_id JOIN asset_classes ON asset_classes.id = dataset_asset_class_1.asset_class_id 
WHERE asset_classes.name = "Real Estate" AND datasets.id = datasets_2.id)) AND ((EXISTS (SELECT 1 
FROM datasets AS datasets_3 JOIN dataset_maintainer AS dataset_maintainer_1 ON datasets_3.id = dataset_maintainer_1.dataset_id JOIN users ON users.id = dataset_maintainer_1.user_id 
WHERE users.username = "user1" AND datasets.id = datasets_3.id)) OR (EXISTS (SELECT 1 
FROM datasets AS datasets_4 JOIN dataset_maintainer AS dataset_maintainer_2 ON datasets_4.id = dataset_maintainer_2.dataset_id JOIN users ON users.id = dataset_maintainer_2.user_id 
WHERE users.username = "user2" AND datasets.id = datasets_4.id)))


   sqlite:///mytable.db
 * sqlite:///test_database.db
Done.


id,parent,notused,detail
2,0,0,SCAN datasets
5,0,0,CORRELATED SCALAR SUBQUERY 1
11,5,0,SEARCH datasets_1 USING INTEGER PRIMARY KEY (rowid=?)
14,5,0,SCAN regions
20,5,0,BLOOM FILTER ON dataset_region_1 (region_id=? AND dataset_id=?)
30,5,0,SEARCH dataset_region_1 USING AUTOMATIC COVERING INDEX (region_id=? AND dataset_id=?)
43,0,0,CORRELATED SCALAR SUBQUERY 2
49,43,0,SEARCH datasets_2 USING INTEGER PRIMARY KEY (rowid=?)
52,43,0,SCAN asset_classes
58,43,0,BLOOM FILTER ON dataset_asset_class_1 (asset_class_id=? AND dataset_id=?)
